# Prédiction des prix de l'immobilier à Boston dans les années 1970

La prédiction du prix de maisons bostoniennes des années 1970, dont les données sont issues de la base *Boston House Prices*, créée par D. Harrison et D.L. Rubinfeld à l'Université de Californie à Irvine (http://archive.ics.uci.edu/ml/machine-learning-databases/housing/), est un problème classique d'apprentissage supervisé.

<img src="https://1.bp.blogspot.com/-sCZIatDf9LQ/XGm-lEHXnAI/AAAAAAAAPxQ/kv8S8fdgudAwWTFuJhuAoiykLmWLCoOtgCLcBGAs/s1600/197010xx-GovernmentCenter-Boston_resize.JPG" width=600 />

Plus précisément, le label à prédire dans cette base de données est le prix médian par quartier de l'immobilier (en milliers de dollars). Il s'agit donc d'un problème de régression puisque l'on veut inférer des valeurs continues. Pour ce faire, on dispose de 13 entrées offrant les informations suivantes :

- CRIM - per capita crime rate by town
- ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
- INDUS - proportion of non-retail business acres per town.
- CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
- NOX - nitric oxides concentration (parts per 10 million)
- RM - average number of rooms per dwelling
- AGE - proportion of owner-occupied units built prior to 1940
- DIS - weighted distances to five Boston employment centres
- TAX - full-value property-tax rate per \$10,000
- RAD - index of accessibility to radial highways
- PTRATIO - pupil-teacher ratio by town
- B $ = 1000(B_k - 0.63)^2$ where $B_k$ is the proportion of blacks by town
- LSTAT - percentage lower status of the population

L'objectif de ce TP est d'arriver à prédire au plus près les valeurs médianes de prix de maison par quartier.


![Texte alternatif…](https://miro.medium.com/max/763/1*i9vZk7NkS1dZz6JEcbV5nA.png)

In [ ]:
from __future__ import print_function
import numpy as np
from matplotlib import pyplot as plt
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers

%matplotlib inline

##### _**Exercice** : Définir une fonction d'affichage `plot_loss` qui permet d'afficher erreur d'entraînement et de validation._

On pourra tracer les courbes associées aux erreurs d'entraînement/de validation par epoch. 

In [1]:
# %load solutions/plot_loss.py
def plot_loss(val_loss, train_loss, ymax=100):
    plt.plot(val_loss, color='green', label='Erreur de validation')
    plt.plot(train_loss, color='blue', linestyle='--', label='Erreur d\'entraînement')
    plt.xlabel('Epochs')
    plt.ylim(0, ymax)
    plt.title('Évolution de la perte sur les ensembles d\'apprentissage et de validation au cours de l\'apprentissage')
    plt.legend()

## Préparation des données

On commence par charger les données d'entraînement et de test.

In [ ]:
from tensorflow.keras.datasets import boston_housing
from sklearn.model_selection import train_test_split

In [ ]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

## Approche simple à corriger

Nous allons commencer par créer un perceptron multicouche élementaire.

### Création du modèle

La fonction `Sequential` permet d'instancier un réseau de neuronnes, la fonction `add` permet d'ajouter une couche au réseau, enfin la fonction `Dense` correspond à un perceptron (monocouche).

In [ ]:
model = Sequential()
model.add(Dense(4, activation='relu', input_dim=13))
model.add(Dense(1, activation='sigmoid'))

### Entrainement du réseau

La fonction `compile` permet de passer les arguments nécessaires à l'entraînement du réseau. `history` stocke les calculs de la loss pour chacune des epochs.

In [ ]:
optim = optimizers.SGD(lr = 0.01)
model.compile(optimizer=optim, loss='mse', metrics=['mae'])

history = model.fit(x_train, y_train, epochs=50, batch_size=32)

### Evaluation du modèle

In [ ]:
train_loss=(history.history['loss'])
plot_loss([], train_loss, ymax=800)

In [ ]:
model.evaluate(x_test, y_test)

On obtient une mae d'environ 22, ce qui signie que l'on est éloigné en moyenne de 22000$ de la vérité terrain.

## Travail à faire

L'approche présentée ci-dessus apporte des résultats décevants, en raison de  quelques maladresses, voire erreurs. Dans un premier temps, vous devez **trouver et corriger ces problèmes**.

Dans un second temps, cherchez à améliorer les performances du modèle. Vous pouvez atteindre sans trop de difficulté un score de MAE inférieur à 3 sur l'ensemble de test. A chaque nouveau test, vous devez évaluer si votre réseau est en sous-apprentissage, ou en sur-apprentissage, et en déduire des modifications possibles pour en améliorer les performances.

MAE de test à battre si vous aimez les défis : **2.20** !

### Correction du modèle précédent

Vous penserez à évaluer votre modèle à l'aide de la fonction `plot_loss` définie précédement et de la fonction `evaluate`.

In [2]:
# %load solutions/correction.py
# Création d'un ensemble de validation
(x, y), (x_test, y_test) = boston_housing.load_data()
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=1/10, random_state=2)

# Activation linéaire sur la couche de sortie
model = Sequential()
model.add(Dense(4, activation='relu', input_dim=13))
model.add(Dense(1, activation='linear'))

optim = optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optim, loss='mse', metrics=['mae'])

# Calcul de l'erreur de validation au cours de l'optimisation
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=32)

In [3]:
# %load solutions/evaluation.py
# Evaluation du modèle
train_loss=(history.history['mae'])
val_loss=(history.history['val_mae'])
plot_loss(val_loss, train_loss, ymax=30)

model.evaluate(x_test, y_test)

### Perceptron monocouche

Obtient-on des résultats comparables au réseau précédent avec un perceptron monocouche ?

In [4]:
# %load solutions/monocouche.py
model = Sequential()
model.add(Dense(1, activation='linear', input_dim=13))

optim = optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optim, loss='mse', metrics=['mae'])

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=16)

In [ ]:
# Evaluation du modèle
train_loss=(history.history['mae'])
val_loss=(history.history['val_mae'])
plot_loss(val_loss, train_loss, ymax=30)

model.evaluate(x_test, y_test)

## Propositions d'améliorations

On peut certainement trouver de meilleures architectures !

### Amélioration n°1

In [5]:
# %load solutions/amelioration_1.py
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=13))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

optim = optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optim, loss='mse', metrics=['mae'])

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=32)

In [ ]:
# Evaluation du modèle
train_loss=(history.history['mae'])
val_loss=(history.history['val_mae'])
plot_loss(val_loss, train_loss, ymax=30)

model.evaluate(x_test, y_test)

### Amélioration n°2

In [6]:
# %load solutions/amelioration_2.py
(x, y), (x_test, y_test) = boston_housing.load_data()
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=1/10, random_state=2)

# Normalisation des entrées
x_mean = np.mean(x_train, axis=0)
x_std = np.std(x_train, axis=0)

x_train = (x_train-x_mean)/x_std
x_val = (x_val-x_mean)/x_std
x_test = (x_test-x_mean)/x_std

print(x_train)
print(x_std)


# ------ #
print(" ")
print(" ")
# ------ #


model = Sequential()
model.add(Dense(100, activation='relu', input_dim=13))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))


optim = optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optim, loss='mse', metrics=['mae'])

history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=32)

In [ ]:
# Evaluation du modèle
train_loss=(history.history['mae'])
val_loss=(history.history['val_mae'])
plot_loss(val_loss, train_loss, ymax=30)

model.evaluate(x_test, y_test)